### Assignment
1. Go through this Post: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/ (Links to an external site.) <br>
2. Add these improvements to the final code described in the post: <br>
3. Predict 500 characters only <br>
4. Remove all the punctuation from the source text <br>
5. Train the model on padded sequences (Links to an external site.) rather than random sequences of characters.  <br>
6. Train the model for 100 epochs <br>
7. Add dropout to the input layer, remove it from the layer before dense layer. Use Dropout value of 0.1 everywhere 
8. Submit! <br>

In [0]:
# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [135]:
# Mount google drive to store check point model results.
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Specify gdrive path to load input file
import os 
filename = "/content/gdrive/My Drive/EIP3/wonderland.txt"

In [0]:
# load ascii text and covert to lowercase
#filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()



### Remove punctuation

In [142]:
# define punctuation
punctuations = '''`!()-[]{};:'"\,<>./?@#$%^&*_~'''

#my_str = "Hello!!!, he said ---and went."

# To take input from the user
# my_str = input("Enter a string: ")

def remove_punc(raw_text):
  # remove punctuation from the string
  no_punct = ""
  for char in raw_text:
    if char not in punctuations:
      no_punct = no_punct + char
  return no_punct
# display the unpunctuated string
#print(no_punct)

raw_text = remove_punc(raw_text)
print raw_text[:1000]

alices adventures in wonderland

                alices adventures in wonderland

                          lewis carroll

               the millennium fulcrum edition 30




                            chapter i

                      down the rabbithole


  alice was beginning to get very tired of sitting by her sister
on the bank and of having nothing to do  once or twice she had
peeped into the book her sister was reading but it had no
pictures or conversations in it and what is the use of a book
thought alice without pictures or conversation

  so she was considering in her own mind as well as she could
for the hot day made her feel very sleepy and stupid whether
the pleasure of making a daisychain would be worth the trouble
of getting up and picking the daisies when suddenly a white
rabbit with pink eyes ran close by her

  there was nothing so very remarkable in that nor did alice
think it so very much out of the way to hear the rabbit say to
itself oh dear  oh dear  i shall be

In [143]:
raw_text[:100]

'alices adventures in wonderland\n\n                alices adventures in wonderland\n\n                  '

In [144]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab


Total Characters:  140265
Total Vocab:  30


In [145]:
char_to_int
int_to_char


{0: '\n',
 1: ' ',
 2: '0',
 3: '3',
 4: 'a',
 5: 'b',
 6: 'c',
 7: 'd',
 8: 'e',
 9: 'f',
 10: 'g',
 11: 'h',
 12: 'i',
 13: 'j',
 14: 'k',
 15: 'l',
 16: 'm',
 17: 'n',
 18: 'o',
 19: 'p',
 20: 'q',
 21: 'r',
 22: 's',
 23: 't',
 24: 'u',
 25: 'v',
 26: 'w',
 27: 'x',
 28: 'y',
 29: 'z'}

In [111]:

148574-100

148474

### Added Padded sequence with input sequece 

In [146]:
 
from keras.preprocessing.sequence import pad_sequences

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  
  ## padded sequence
  idx = numpy.random.randint(0, seq_length)
  temp_x = ([char_to_int[char] for char in seq_in])
  len_temp_X = len(temp_x)
  temp_X = temp_x[idx:len(temp_x)]
  dataX.append(temp_X)
  dataY.append(char_to_int[seq_out])
  
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])

dataXX = pad_sequences(dataX).tolist()
print type(dataX)
print type(dataXX)
n_patterns = len(dataXX)
print "Total Patterns: ", n_patterns
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataXX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)


<type 'list'>
<type 'list'>
Total Patterns:  280330


### Added dropout 0.1 to input and removed dropout on output layer
### check point of best model stored in gdrive

In [152]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
checkpoint = "/content/gdrive/My Drive/EIP3/assignment6_v3.hdf5"

filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(checkpoint, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=100, batch_size=2048, callbacks=callbacks_list)


Epoch 1/100
280330/280330 [==============================] - 95s 338us/step - loss: 2.8734

Epoch 00001: loss improved from inf to 2.87339, saving model to /content/gdrive/My Drive/EIP3/assignment6_v3.hdf5
Epoch 2/100
280330/280330 [==============================] - 99s 354us/step - loss: 2.8306

Epoch 00002: loss improved from 2.87339 to 2.83057, saving model to /content/gdrive/My Drive/EIP3/assignment6_v3.hdf5
Epoch 3/100
280330/280330 [==============================] - 105s 375us/step - loss: 2.7646

Epoch 00003: loss improved from 2.83057 to 2.76456, saving model to /content/gdrive/My Drive/EIP3/assignment6_v3.hdf5
Epoch 4/100
280330/280330 [==============================] - 106s 379us/step - loss: 2.6176

Epoch 00004: loss improved from 2.76456 to 2.61764, saving model to /content/gdrive/My Drive/EIP3/assignment6_v3.hdf5
Epoch 5/100
280330/280330 [==============================] - 101s 361us/step - loss: 2.4800

Epoch 00005: loss improved from 2.61764 to 2.48004, saving model to /

In [154]:

# pick a random seed
start = numpy.random.randint(0, len(dataXX)-1)
pattern = dataXX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
  #patten.concatinate(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
"  another hedgehog
which seemed to alice an excellent opportunity for croqueting one
of them with the "
 other  the only difficulty was that her
flamingo mike a serpent wiich sald the cortth 
  the fant ruite licker nutt be the said this she said in a hoaring anowh
eeil hfr foeam of the senew ane denne sn the things beooss the gande
   i dadalt begin thich wpulder as she hett to gnennny down hye held

  she moment she waid to herself as she ran while she was sallen anynhing beoots in the pepple and tee whth all the restent said and were ouierfly    aying nother afoue as she mefth gersilg sp co so 
Done.


#### Conclusion
* Have implemented larger LSTM Recurrent Neural Network <br>
* Removed punctuation from Input data set.<br>
* Added dropout 0.1 to input layer and removed dropout from output layer.<br>
* Total number of sequence in the input data is  1,48,474 and after added a one padded sequence for each sequence in each in the input layer, total number of sequence is 2,80,330 <br>
* Have generated a random seed and made model to predict next 500 character in that sequence.